# <center>Деревья решений

На самом деле деревья решений встречаются в повседневной жизни довольно часто, например, в виде инструкций. Вот пример такой инструкции к действию при поломке чего либо:

<br><img src="https://c1.staticflickr.com/9/8160/7214525854_733237dd83_z.jpg">

Схему, стояющую выше можно интерпретировать как функцию (грубо говоря), где на вход подается предмет, а на выходе мы получаем указания по взаимодействию с ним.
<br><br>
**F(заело велоцепь) = WD-40**
<br>
**F(оторвался бампер) = армированный скотч**
<br>
<br>
Мы познакомились с примитивной схемой решения проблем инженеров на производстве, а что если попробовать создать схему для решения проблем data scientist-ов? Тогда мы получим дерево решений. Только вместо кучи неисправного хлама у нас есть стек объектов (матрица <font color='blue'>X</font>) и значения целевой переменной для каждого из объектов (вектор <font color='blue'>y</font>) вместо указаний по взаимодействию. Так попробуем найти такую функцию (то есть построить такое дерево) которое бы принимало объект и выдавало на выходе значение целевого признака. Но как ее строить? Разумеется на основе имеющихся данных, а потом лишь надеяться, что эта функция будет работать и в будущем, на новых объектах. Но как уменьшить ошибку в будущем? Не дать дереву "переобучиться", но при этом и "обучить" достаточно хорошо, то есть придется искать золотую середину между двумя этими состояниями.

#### Рассмотрим еще один пример из банковской сферы
<br>
Предположим надо создать систему, которая определяла бы по определенным критериям, выдавать ли кредит человеку или нет, основываясь на богатом опыте успешно возвращенных кредитов и потерь. Возможно, она могла бы выглядеть примерно так:
<img src="https://habrastorage.org/files/194/9b6/ae9/1949b6ae97ab4fc9b1a37fbf182eda8f.gif"/><br>

- **Приведите пример бинарного признака, который мог бы также появиться в этой схеме**
- **Приведите пример численного признака**
- **Какой признак лучше всего разделяет выборку?**

Алгоритм построения дерева можно представить в виде рекурсивной функции:

```python
def build(L):
    create node t
    if the stopping criterion is True:
        assign a predictive model to t
    else:
        Find the best binary split L = L_left + L_right
        t.left = build(L_left)
        t.right = build(L_right)
    return t     
```

Со всеми строками кроме одной все более менее понятно, а именно вызывает вопросы
```python
    Find the best binary split L = L_left + L_right
```
Что есть "the best binary split" - лучшее бинарное разбиение? Это разбиение, при котором новые подгруппы лучше всего разделены по целевому признаку. Например, split клиентов банка по возрасту лучше всего разделяет потенциальных должников и богачей.

#### И крайний графический пример, на этот раз максимально искусственный

<img src="img/1DT.png"/><br>

<img src="img/2DT.png"/><br>

<img src="img/3DT.png"/><br>

<img src="img/4DT.png"/><br>

<img src="img/5DT.png"/><br>

#### В итоге получили требуемую "функцию" 

<img src="img/resultDT.png">

### <center> Класс DecisionTreeClassifier в Scikit-learn</center>
Основные параметры класса [sklearn.tree.DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html):

- `max_depth` – максимальная глубина дерева
- `max_features` - максимальное число признаков, по которым ищется лучшее разбиение в дереве (это нужно потому, что при большом количестве признаков будет "дорого" искать лучшее (по критерию типа прироста информации) разбиение среди *всех* признаков)
- `min_samples_leaf` – минимальное число объектов в листе. У этого параметра есть понятная интерпретация: скажем, если он равен 5, то дерево будет порождать только те классифицирующие правила, которые верны как мимимум для 5 объектов

Параметры дерева надо настраивать в зависимости от входных данных, и делается это обычно с помощью *кросс-валидации*,  про нее, возможно, позже: на следующих занятиях.

----

Рассмотрим искусственный пример для знакомства с реализацией дерева решений

In [1]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
data = pd.DataFrame({'Возраст': [17,64,18,20,38,49,55,25,29,31,33], 
             'Невозврат кредита': [1,0,1,0,1,0,0,1,1,0,1]})

In [3]:
data 

,Возраст,Невозврат кредита
0,17,1
1,64,0
2,18,1
3,20,0
4,38,1
5,49,0
6,55,0
7,25,1
8,29,1
9,31,0


In [4]:
data.sort_values('Возраст')

,Возраст,Невозврат кредита
0,17,1
2,18,1
3,20,0
7,25,1
8,29,1
9,31,0
10,33,1
4,38,1
5,49,0
6,55,0


Обучим дерево решений на нашем искусственном примере и посмотрим на него

In [5]:
age_tree = DecisionTreeClassifier(random_state=17, max_depth=3)
age_tree.fit(data['Возраст'].values.reshape(-1, 1), data['Невозврат кредита'].values)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [6]:
export_graphviz(age_tree, feature_names=['Возраст'], 
                out_file='./age_tree.dot', filled=True)

########################################################################################################
##   Терминальный команды, используемые далее, работают только в Unix-подобных операционных системах  ##
##   для windows можно скачать приложение graphviz и открыть файл './cardio_tree_hw3.dot' в нем.      ##
########################################################################################################

!dot -Tpng ./age_tree.dot -o ./age_tree.png

<img src='./age_tree.png'>

_____

<font size=1>Автор: Тимур Фатыхов. <br>
По материалам программиста-исследователя Mail.ru Group, старшего преподавателя Факультета Компьютерных Наук ВШЭ Юрия Кашницкого. 